In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
	# create model
	model = Sequential()
	model.add(Dense(16, input_dim=13, activation='linear', kernel_initializer='uniform', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))
	# Compile model
	#optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
dataset = pd.read_csv("slim-xAPI-Edu-Data.csv")
# split into input (X) and output (Y) variables

X = dataset


#y = dataset[['Class']]
#students failed if they are in class L.
dataset['Failed'] = np.where(dataset['Class'] == 'L', True, False)
dataset['gender'] = np.where(dataset['gender']=='M',1,0)
dataset['Relation'] = np.where(dataset['Relation']=='Father',1,0)
dataset['ParentAnsweringSurvey'] = np.where(dataset['ParentAnsweringSurvey'] == 'Yes', 1, 0)
dataset['ParentschoolSatisfaction'] = np.where(dataset['ParentschoolSatisfaction'] == 'Yes', 1, 0)
dataset['AbsentMoreThanWeek'] = np.where(dataset['StudentAbsenceDays'] == 'Above-7', 1, 0)
dataset['Semester'] = np.where(dataset['Semester'] == 'F', 1, 0)

#SectionID: 1=A, 2=B, 3=C
#Topic: 1=Arabic, 2=Biology, 3=Chemistry, 4=English, 5=French, 6=Geology, 7=History, 8=IT, 9=Math, 10=Quran, 11=Science, 12=Spanish
#StageID: Lowerlevel = 1, MiddleSchool = 2, HighSchool = 3
X = dataset[['raisedhands', 'VisITedResources', 'SectionID', 'Topic', 'StageID', 'AnnouncementsView', 'Semester', 'Discussion', 'gender', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'AbsentMoreThanWeek']]

Y = dataset[['Failed']]




In [8]:

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=10, epochs=100)
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# define the grid search parameters
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.914583 using {'dropout_rate': 0.4, 'weight_constraint': 2}
0.885417 (0.028106) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.881250 (0.023385) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.904167 (0.007795) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.891667 (0.010623) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.893750 (0.013502) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.908333 (0.010623) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.900000 (0.013502) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.910417 (0.016404) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.910417 (0.007795) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.912500 (0.013502) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.912500 (0.013502) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.906250 (0.008839) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.912500 (0.008839) with: {'dropout_rate': 0.2, 'weight_constraint': 